In [0]:
pip install plotly

In [0]:
import pandas as pd
from google.colab import files
import io
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as md
import seaborn as sns
import plotly.express as px

In [0]:
uploaded = files.upload()

Saving screen_login_cleaned - Screen_Login.csv to screen_login_cleaned - Screen_Login.csv


In [0]:
data = pd.read_csv(io.BytesIO(uploaded['screen_login_cleaned - Screen_Login.csv']))

In [0]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
Screens                50 non-null object
Screen 2               42 non-null object
Screen Name            50 non-null object
login status           50 non-null object
Screen Views           50 non-null object
Unique Screen Views    50 non-null object
Avg. Time on Screen    50 non-null object
% Exit                 50 non-null object
dtypes: object(8)
memory usage: 3.2+ KB


In [0]:
df = data.iloc[:,2:]
df.columns = 'screen_name', 'login_status', 'screen_views', 'unique_screen_views', 'avg_time_on_screen', 'exits'
df = df.reset_index(drop='index')

In [0]:
li_1 = []
li_2 = []
li_time = []

for i in df.index:
  try: 
    li_1.append(df.screen_name[i].split(' /')[0])
    li_2.append(df.screen_name[i].split('/ ')[1])
    li_time.append(datetime.strptime(df.avg_time_on_screen[i], "%H:%M:%S").strftime("%H:%M:%S"))
  except:
    li_2.append('None')

df['screens'] = li_1
df['streams'] = li_2
df['avg_time'] = li_time

In [0]:
df['screenviews'] = list(map(lambda x: int(float(x)), df.screen_views.str.replace(",", "")))
df['u_screenviews'] = list(map(lambda x: int(float(x)), df.unique_screen_views.str.replace(",", "")))

In [0]:
time_seconds = []

for row in df.index:
  ftr = [3600,60,1]
  time_seconds.append(sum([a*b for a,b in zip(ftr, [int(i) for i in df.avg_time[row].split(":")])]))

In [0]:
data_clean = df.drop(['screen_views', 'unique_screen_views', 'avg_time_on_screen'], axis=1)
data_clean['time_seconds'] = time_seconds

In [0]:
data_clean.head()

,screen_name,login_status,exits,screens,streams,screenviews,u_screenviews,avg_time,time_seconds
0,home,login,23.61%,home,None,366130994,138141703,00:01:08,68
1,home,not_login,24.66%,home,None,204006177,81588074,00:01:05,65
2,livestreaming / SCTV TV Stream,login,8.90%,livestreaming,SCTV TV Stream,119832291,47476545,00:05:39,339
3,livestreaming / Indosiar TV Stream,login,8.76%,livestreaming,Indosiar TV Stream,96600949,40363705,00:05:20,320
4,livestreaming / RCTI TV Stream,login,8.19%,livestreaming,RCTI TV Stream,65973859,31293331,00:04:42,282


In [0]:
plot_login = data_clean[['screens', 'streams', 'screenviews', 'time_seconds', 'login_status']]

In [0]:
ax = plt.figure(figsize=(16, 8))
ax = px.area(plot_login, x='time_seconds', y='screenviews', color="screens")
ax.show()

<Figure size 1152x576 with 0 Axes>

In [0]:
ax = plt.figure(figsize=(16, 4))
ax = px.histogram(plot_login, x='time_seconds', y='screenviews', color="screens", opacity=0.8)
ax.show()

<Figure size 1152x288 with 0 Axes>

In [0]:
fig = px.scatter_polar(plot_login, r="time_seconds", theta="streams", color="login_status", symbol="screens")
fig.show()